<a href="https://colab.research.google.com/github/arunraja-hub/Preference_Extraction/blob/fine_tune/fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install and imports

In [1]:
%tensorflow_version 1.x

!git clone https://github.com/arunraja-hub/Preference_Extraction.git

!pip install tf-agents==0.3.0

!pip uninstall tensorflow-probability -y
!pip install tensorflow-probability==0.7.0

import tensorflow as tf

tf.compat.v1.disable_eager_execution()

TensorFlow 1.x selected.
Cloning into 'Preference_Extraction'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 652 (delta 0), reused 0 (delta 0), pack-reused 649
Receiving objects: 100% (652/652), 21.38 MiB | 14.40 MiB/s, done.
Resolving deltas: 100% (86/86), done.
     |████████████████████████████████| 839kB 8.6MB/s 
     |████████████████████████████████| 51kB 8.4MB/s 
  Found existing installation: gin-config 0.3.0
    Uninstalling gin-config-0.3.0:
      Successfully uninstalled gin-config-0.3.0
Uninstalling tensorflow-probability-0.7.0:
  Successfully uninstalled tensorflow-probability-0.7.0
     |████████████████████████████████| 983kB 8.3MB/s 
  Found existing installation: tensorflow-probability 0.10.0rc0
    Uninstalling tensorflow-probability-0.10.0rc0:
      Successfully uninstalled tensorflow-probability-0.10.0rc0


In [0]:
import numpy as np

from tf_agents.trajectories.time_step import TimeStep
from tf_agents.specs.tensor_spec import TensorSpec
from tf_agents.specs.tensor_spec import TensorSpec
from tf_agents.specs.tensor_spec import BoundedTensorSpec
from tf_agents.networks import q_network

import concurrent.futures
import itertools
import os
import pickle
import random
import sys
import time

import numpy as np
import io
import collections

import urllib.request
from urllib.error import HTTPError

# SetUp Model

In [0]:
cpt_name = "Preference_Extraction/model_ckpt"

In [4]:
cpt_var_names = tf.compat.v1.train.list_variables(cpt_name)
tf.reset_default_graph()

input_shape = [14, 16, 5]
my_input = tf.placeholder(tf.float32, shape=[None] + input_shape, name="my_input")
q_vals = q_network.QNetwork(input_tensor_spec=TensorSpec(shape=(14, 16, 5)), action_spec=BoundedTensorSpec((), tf.int32, 0, 2), conv_layer_params = [[16, 3, 1], [32, 3, 2]], fc_layer_params = [64])(my_input)

cpt_var_info = tf.compat.v1.train.list_variables(cpt_name)
cpt_var_info = [var for var in cpt_var_info if (("bias" in var[0]) or ("kernel" in var[0])) and not ("OPTIMIZER" in var[0]) and not ("_target_q_network" in var[0])]
shape_to_cpt_var_name = {tuple(var[1]): var[0] for var in cpt_var_info}

current_vars = tf.get_collection(tf.GraphKeys.VARIABLES)
shape_to_current_var_name = {tuple(var.get_shape().as_list()): var.name[:-2] for var in current_vars}

var_name_to_prev_var_name = {}

for shape in shape_to_current_var_name:
  var_name_to_prev_var_name[shape_to_current_var_name[shape]] = shape_to_cpt_var_name[shape]

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use `tf.GraphKeys.GLOBAL_VARIABLES` instead.


# Read data

In [0]:
class Trajectory(
    collections.namedtuple('Trajectory', [
        'step_type',
        'observation',
        'action',
        'policy_info',
        'next_step_type',
        'reward',
        'discount',
    ])):
  """Stores the observation the agent saw and the action it took.
      The rest of the attributes aren't used in this code."""
  __slots__ = ()

class ListWrapper(object):
  def __init__(self, list_to_wrap):
    self._list = list_to_wrap

  def as_list(self):
    return self._list

class RenameUnpickler(pickle.Unpickler):
    def find_class(self, module, name):
      if name == "Trajectory":
        return Trajectory
      if name == "ListWrapper":
        return ListWrapper

      return super(RenameUnpickler, self).find_class(module, name)

def rename_load(s):
    """Helper function analogous to pickle.loads()."""
    return RenameUnpickler(s, encoding='latin1').load()

In [0]:
# Modified read trajectories functions to read files from local storage

def load_file(full_path):
    try:
        with open(full_path, 'rb') as f:
            data = rename_load(f)
            return data
    except:
        return None
    
def all_load_data(base_path):
    
    executor = concurrent.futures.ThreadPoolExecutor(max_workers=100)
    
    futures = []
    for i in range(5000):
        full_path = os.path.join(base_path, "ts"+str(i)+".pickle")
        future = executor.submit(load_file, full_path)
        futures.append(future)
    
    raw_data = []
    for future in concurrent.futures.as_completed(futures):
        result = future.result()
        if result:
            raw_data.append(result)
    
    return raw_data


all_raw_data = all_load_data("Preference_Extraction/data/simple_env_1/")

# Do the warmstart and verify it worked

In [7]:
tf.train.warm_start(cpt_name, var_name_to_prev_var_name=var_name_to_prev_var_name)
init_op = tf.global_variables_initializer()

INFO:tensorflow:Warm-starting from: Preference_Extraction/model_ckpt
INFO:tensorflow:Warm-starting variables only in TRAINABLE_VARIABLES.
INFO:tensorflow:Warm-started 8 variables.


In [9]:
with tf.Session() as sess:
  sess.run(init_op)
  
  graph = tf.get_default_graph()
  print(graph)